In [ ]:
from pathlib import Path
import json

# Notebook içeriği
notebook_content = {
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# 🚗 Car Evaluation Dataset ile Özel Mesafe Metriği Karşılaştırması\n",
                "Bu örnekte, özel mesafe metriğiyle donatılmış KNN modeli, klasik Euclidean metrik ile karşılaştırılmaktadır."
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Gerekli importlar\n",
                "import numpy as np\n",
                "import pandas as pd\n",
                "from sklearn.model_selection import StratifiedKFold\n",
                "from sklearn.neighbors import KNeighborsClassifier\n",
                "from sklearn.metrics import accuracy_score, classification_report\n",
                "from custom_knn_distance import (\n",
                "    create_custom_distance,\n",
                "    load_and_preprocess_car_data,\n",
                "    calculate_specificity,\n",
                "    calculate_average_reports,\n",
                "    reports_to_dataframe\n",
                ")"
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Veri setini yükle\n",
                "X_encoded, y, classes = load_and_preprocess_car_data()\n",
                "X = X_encoded"
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# K-Fold ve metrik listeleri\n",
                "kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)\n",
                "cv_scores_euclidean = []\n",
                "cv_reports_euclidean = []\n",
                "cv_specificities_euclidean = {label: [] for label in classes}\n",
                "cv_scores_custom = []\n",
                "cv_reports_custom = []\n",
                "cv_specificities_custom = {label: [] for label in classes}"
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# K-Fold döngüsü\n",
                "for train_idx, test_idx in kf.split(X, y):\n",
                "    X_train, X_test = X[train_idx], X[test_idx]\n",
                "    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]\n",
                "    \n",
                "    # Euclidean\n",
                "    knn_euc = KNeighborsClassifier(n_neighbors=3, metric='euclidean')\n",
                "    knn_euc.fit(X_train, y_train)\n",
                "    preds_euc = knn_euc.predict(X_test)\n",
                "    cv_scores_euclidean.append(accuracy_score(y_test, preds_euc))\n",
                "    cv_reports_euclidean.append(classification_report(y_test, preds_euc, output_dict=True))\n",
                "    for label, spec in calculate_specificity(y_test, preds_euc, classes).items():\n",
                "        cv_specificities_euclidean[label].append(spec)\n",
                "    \n",
                "    # Custom\n",
                "    custom_metric = create_custom_distance(X_train)\n",
                "    knn_custom = KNeighborsClassifier(n_neighbors=3, metric=custom_metric)\n",
                "    knn_custom.fit(X_train, y_train)\n",
                "    preds_custom = knn_custom.predict(X_test)\n",
                "    cv_scores_custom.append(accuracy_score(y_test, preds_custom))\n",
                "    cv_reports_custom.append(classification_report(y_test, preds_custom, output_dict=True))\n",
                "    for label, spec in calculate_specificity(y_test, preds_custom, classes).items():\n",
                "        cv_specificities_custom[label].append(spec)"
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Sonuçlar\n",
                "print(f\"Ortalama doğruluk (Euclidean): {np.mean(cv_scores_euclidean)}\")\n",
                "print(f\"Ortalama doğruluk (Custom): {np.mean(cv_scores_custom)}\")"
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Ortalama raporlar\n",
                "df_euc = reports_to_dataframe(calculate_average_reports(cv_reports_euclidean))\n",
                "df_spec_euc = pd.DataFrame({k: [np.mean(v)] for k, v in cv_specificities_euclidean.items()}).T\n",
                "df_spec_euc.columns = [\"Specificity\"]\n",
                "df_cus = reports_to_dataframe(calculate_average_reports(cv_reports_custom))\n",
                "df_spec_cus = pd.DataFrame({k: [np.mean(v)] for k, v in cv_specificities_custom.items()}).T\n",
                "df_spec_cus.columns = [\"Specificity\"]"
            ]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Görüntüle\n",
                "display(df_euc)\n",
                "display(df_spec_euc)\n",
                "display(df_cus)\n",
                "display(df_spec_cus)"
            ]
        }
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "Python 3",
            "language": "python",
            "name": "python3"
        },
        "language_info": {
            "name": "python",
            "version": "3.9"
        }
    },
    "nbformat": 4,
    "nbformat_minor": 5
}

# Dosya olarak kaydet
notebook_file_path = Path("/mnt/data/car_demo_notebook.ipynb")
notebook_file_path.write_text(json.dumps(notebook_content, indent=2), encoding="utf-8")

notebook_file_path.name